# Tissue comparison for differential expression analysis

In [ ]:
import functools
import numpy as np
import pandas as pd
from gtfparse import read_gtf

In [ ]:
config = {
    'caudate': '../../../caudate/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt',
    'dlpfc': '../../../dlpfc/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt',
    'hippo': '../../../hippocampus/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt',
    'cmc_dlpfc': '../../../cmc_dlpfc/male_analysis/metrics_summary/_m/male_specific_DE_genes.txt'
}

In [ ]:
@functools.lru_cache()
def get_gtf(gtf_file):
    return read_gtf(gtf_file)


@functools.lru_cache()
def get_deg(filename):
    dft = pd.read_csv(filename, sep='\t', index_col=0)
    dft = dft[(dft['Type'] == 'gene')].copy()
    dft['Feature'] = dft.index
    dft['Dir'] = np.sign(dft['t'])
    if 'gene_id' in dft.columns:
        dft['ensemblID'] = dft.gene_id.str.replace('\\..*', '', regex=True)
    return dft[['Feature', 'ensemblID', 'adj.P.Val', 'logFC', 't', 'Dir']]

@functools.lru_cache()
def get_deg_sig(filename):
    dft = get_deg(filename)
    return dft[(dft['adj.P.Val'] < 0.05)]


@functools.lru_cache()
def merge_dataframes(tissue1, tissue2):
    return get_deg(config[tissue1]).merge(get_deg(config[tissue2]), 
                                          on='Feature', 
                                          suffixes=['_%s' % tissue1, '_%s' % tissue2])


@functools.lru_cache()
def merge_dataframes_sig(tissue1, tissue2):
    return get_deg_sig(config[tissue1]).merge(get_deg_sig(config[tissue2]), 
                                              on='Feature', 
                                              suffixes=['_%s' % tissue1, '_%s' % tissue2])

In [ ]:
def tissue_annotation(tissue):
    return {'dlpfc': 'DLPFC', 'hippo': 'Hippocampus', 
            'caudate': 'Caudate', 'cmc_dlpfc': 'CMC DLPFC'}[tissue]


def save_plot(p, fn, width=7, height=7):
    '''Save plot as svg, png, and pdf with specific label and dimension.'''
    for ext in ['.svg', '.png', '.pdf']:
        p.save(fn+ext, width=width, height=height)
        

def gene_annotation(gtf_file, feature):
    gtf0 = get_gtf(gtf_file)
    gtf = gtf0[gtf0["feature"] == feature]
    return gtf[["gene_id", "gene_name", "transcript_id", "exon_id", "gene_type", 
                "seqname", "start", "end", "strand"]]

## BrainSeq Comparison

In [ ]:
caudate = get_deg(config['caudate'])
caudate.groupby('Dir').size()

In [ ]:
caudate[(caudate['adj.P.Val'] < 0.05)].shape

In [ ]:
dlpfc = get_deg(config['dlpfc'])
dlpfc.groupby('Dir').size()

In [ ]:
dlpfc[(dlpfc['adj.P.Val'] < 0.05)].shape

In [ ]:
hippo = get_deg(config['hippo'])
hippo.groupby('Dir').size()

In [ ]:
hippo[(hippo['adj.P.Val'] < 0.05)].shape

### Upset Plot

In [ ]:
phase2_dlpfc = dlpfc[(dlpfc['adj.P.Val'] < 0.05)].copy()
phase2_dlpfc['DLPFC'] = 1
phase2_dlpfc = phase2_dlpfc[['ensemblID', 'DLPFC']]

phase2_hippo = hippo[(hippo['adj.P.Val'] < 0.05)].copy()
phase2_hippo['Hippocampus'] = 1
phase2_hippo = phase2_hippo[['ensemblID', 'Hippocampus']]

phase3_caudate = caudate[(caudate['adj.P.Val'] < 0.05)].copy()
phase3_caudate['Caudate'] = 1
phase3_caudate = phase3_caudate[['ensemblID', 'Caudate']]

In [ ]:
geneList = pd.merge(phase3_caudate[['ensemblID']], phase2_dlpfc[['ensemblID']], 
                    on=['ensemblID'], how='outer')\
             .merge(phase2_hippo[['ensemblID']], on=['ensemblID'], how='outer')\
             .groupby(['ensemblID']).first().reset_index()

newC = pd.merge(geneList, phase3_caudate, on=['ensemblID'], how='outer').fillna(0)
newC['Caudate'] = newC['Caudate'].astype('int')

newD1 = pd.merge(geneList, phase2_dlpfc, on=['ensemblID'], how='outer').fillna(0)
newD1['DLPFC'] = newD1['DLPFC'].astype('int')

newH = pd.merge(geneList, phase2_hippo, on=['ensemblID'], how='outer').fillna(0)
newH['Hippocampus'] = newH['Hippocampus'].astype('int')

print(newC.shape, newH.shape, newD1.shape)

In [ ]:
df = pd.concat([newC.set_index(['ensemblID']), newD1.set_index(['ensemblID']), 
                newH.set_index(['ensemblID'])], axis=1, join='outer')
df.head(2)

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
library(ComplexHeatmap)
library(tidyverse)
subset_pvalue <- function(filename, fdr_cutoff){
    df <- data.table::fread(filename) %>% 
        filter(Type == 'gene', adj.P.Val < fdr_cutoff)
    return(df$ensemblID)
}

caudate = subset_pvalue('../../../caudate/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt', 
                        0.05)
dlpfc = subset_pvalue('../../../dlpfc/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt', 
                      0.05)
hippo = subset_pvalue('../../../hippocampus/male_analysis/metrics_summary/_m/male_specific_DE_4features.txt',
                      0.05)

lt = list(Caudate = caudate,
          DLPFC = dlpfc,
          Hippocampus = hippo)

m = make_comb_mat(lt)
cbb_palette <- c("#000000", "#E69F00", "#56B4E9", "#009E73", "#F0E442", 
                 "#0072B2", "#D55E00", "#CC79A7")

In [ ]:
%%R
right_annot = upset_right_annotation(
    m, ylim = c(0, 2000),
    gp = gpar(fill = "black"),
    annotation_name_side = "top",
    axis_param = list(side = "top"))

top_annot = upset_top_annotation(
    m, height=unit(7, "cm"), 
    ylim = c(0, 2000),
    gp=gpar(fill=cbb_palette[comb_degree(m)]),
    annotation_name_rot = 90)

pdf('BrainSeq_sex_tissue_upsetR_DEgenes_maleSpecific.pdf', width=8, height=4)
ht = draw(UpSet(m, pt_size=unit(4, "mm"), lwd=3, 
                comb_col=cbb_palette[comb_degree(m)], 
                set_order = c("Caudate", "DLPFC", "Hippocampus"), 
                comb_order = order(-comb_size(m)), 
                row_names_gp = gpar(fontsize = 14, fontface='bold'),
                right_annotation = right_annot, 
                top_annotation = top_annot))
od = column_order(ht)
cs = comb_size(m)
decorate_annotation("intersection_size", {
    grid.text(cs[od], x = seq_along(cs), y = unit(cs[od], "native") + 
              unit(6, "pt"), 
        default.units = "native", just = "bottom", gp = gpar(fontsize = 11))
})
dev.off()

In [ ]:
%%R
right_ha = rowAnnotation(
    "Intersection\nsize" = anno_barplot(comb_size(m), border=F,
                                        ylim = c(0, 2000), 
                                        gp=gpar(fill=cbb_palette[comb_degree(m)]),
                                        width = unit(7, "cm")))
top_ha = HeatmapAnnotation(
    "Set size" = anno_barplot(set_size(m), border=F,
                              ylim = c(0, 2000),
                              gp = gpar(fill = "black"), 
                              height = unit(2, "cm")), 
    gap = unit(2, "mm"), annotation_name_side = "left", 
    annotation_name_rot = 90)


pdf("BrainSeq_sex_tissue_upsetR_DEgenes_transpose_maleSpecific.pdf", width=5, height=10)
ht = draw(UpSet(t(m), pt_size=unit(5, "mm"), lwd=3,
                comb_order = order(-comb_size(m)), 
                comb_col=cbb_palette[comb_degree(m)], 
                set_order = c("Caudate", "DLPFC", "Hippocampus"), 
                column_names_gp = gpar(fontsize = 16, fontface='bold'),
                right_annotation = right_ha, top_annotation=top_ha))

od = rev(row_order(ht))
cs = comb_size(m)
decorate_annotation("Intersection\nsize", {
    grid.text(cs[od], y = seq_along(cs), x = unit(cs[od], "native") + 
              unit(6, "pt"), 
        default.units = "native", just = "left", gp = gpar(fontsize = 11))
})
dev.off()

### Shared features

In [ ]:
gtf_file = '/ceph/genome/human/gencode25/gtf.CHR/_m/gencode.v25.annotation.gtf'
gtf_annot = gene_annotation(gtf_file, 'gene')
gtf_annot.head(2)

In [ ]:
dft = caudate.merge(gtf_annot[['gene_id', 'gene_name', 'seqname']], 
                    left_index=True, right_on='gene_id')
dft.head(2)

In [ ]:
shared_df = dft.loc[:, ['gene_id', 'ensemblID', 'seqname', 'gene_name', 'Dir']]\
               .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                                      set(phase2_hippo['ensemblID']) & 
                                                      set(phase3_caudate['ensemblID']))}), 
                      on='ensemblID')
shared_df.to_csv('BrainSeq_shared_degs_annotation_maleSpecific.txt', 
                 sep='\t', index=False, header=True)
shared_df

In [ ]:
dlpfc.merge(gtf_annot[['gene_id', 'gene_name', 'seqname']], 
            left_index=True, right_on='gene_id')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                            set(phase2_hippo['ensemblID']))}), 
            on='ensemblID')

In [ ]:
dlpfc.merge(gtf_annot[['gene_id', 'gene_name', 'seqname']], 
            left_index=True, right_on='gene_id')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                            set(phase3_caudate['ensemblID']))}), 
            on='ensemblID')

In [ ]:
hippo.merge(gtf_annot[['gene_id', 'gene_name', 'seqname']], 
            left_index=True, right_on='gene_id')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase2_hippo['ensemblID']) & 
                                            set(phase3_caudate['ensemblID']))}), 
            on='ensemblID')

In [ ]:
hippo.merge(gtf_annot[['gene_id', 'gene_name', 'seqname']], 
            left_index=True, right_on='gene_id')\
               .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                                      set(phase2_hippo['ensemblID']))}), 
                      on='ensemblID')

In [ ]:
dd = np.sum(shared_df.seqname.isin(['chrX', 'chrY'])) / shared_df.shape[0] * 100
print("%0.2f%% of shared DEG are allosomal!" % dd)

In [ ]:
gtf_annot['ensemblID'] = gtf_annot.gene_id.str.replace("\\..*", "", regex=True)
gtf_annot[["gene_id", 'ensemblID', 'gene_name', 'seqname', 'gene_type']]\
    .merge(df, left_on='ensemblID', right_index=True)\
    .to_csv('brainseq_deg_across_tissues_comparison_maleSpecific.csv')

## Comparison with CommonMind

In [ ]:
cmc_dlpfc = get_deg(config['cmc_dlpfc'])
cmc_dlpfc.groupby('Dir').size()

In [ ]:
cmc_dlpfc[(cmc_dlpfc['adj.P.Val'] < 0.05)].shape

### Upset Plot

In [ ]:
cmc = cmc_dlpfc[(cmc_dlpfc['adj.P.Val'] < 0.05)].copy()
cmc['CMC DLPFC'] = 1
cmc = cmc[['ensemblID', 'CMC DLPFC']].groupby('ensemblID').first().reset_index()

In [ ]:
geneList = pd.merge(phase3_caudate[['ensemblID']], phase2_dlpfc[['ensemblID']], on=['ensemblID'], how='outer')\
             .merge(phase2_hippo[['ensemblID']], on=['ensemblID'], how='outer')\
             .merge(cmc[['ensemblID']], on=['ensemblID'], how='outer')\
             .groupby(['ensemblID']).first().reset_index()

newC = pd.merge(geneList, phase3_caudate, on=['ensemblID'], how='outer').fillna(0)
newC['Caudate'] = newC['Caudate'].astype('int')

newD1 = pd.merge(geneList, phase2_dlpfc, on=['ensemblID'], how='outer').fillna(0)
newD1['DLPFC'] = newD1['DLPFC'].astype('int')

newH = pd.merge(geneList, phase2_hippo, on=['ensemblID'], how='outer').fillna(0)
newH['Hippocampus'] = newH['Hippocampus'].astype('int')

newCMC = pd.merge(geneList, cmc, on=['ensemblID'], how='outer').fillna(0)
newCMC['CMC DLPFC'] = newCMC['CMC DLPFC'].astype('int')

print(newC.shape, newH.shape, newD1.shape, newCMC.shape)

In [ ]:
df = pd.concat([newC.set_index(['ensemblID']), newD1.set_index(['ensemblID']), 
                newH.set_index(['ensemblID']), newCMC.set_index(['ensemblID'])], axis=1, join='outer')
df.head(2)

In [ ]:
%%R 
cmc = subset_pvalue('../../../cmc_dlpfc/male_analysis/metrics_summary/_m/male_specific_DE_genes.txt',
                    0.05)

lt = list(Caudate = caudate,
          DLPFC = dlpfc,
          Hippocampus = hippo, 
          `CMC DLPFC` = cmc)

m = make_comb_mat(lt)

In [ ]:
%%R
right_annot = upset_right_annotation(
    m, ylim = c(0, 2000),
    gp = gpar(fill = "black"),
    annotation_name_side = "bottom",
    axis_param = list(side = "bottom"))

top_annot = upset_top_annotation(
    m, height=unit(7, "cm"), 
    ylim = c(0, 2000),
    gp=gpar(fill=cbb_palette[comb_degree(m)]),
    annotation_name_rot = 90)

pdf('cmc_sex_tissue_upsetR_DEgenes_maleSpecific.pdf', width=10, height=5)
ht = draw(UpSet(m, pt_size=unit(6, "mm"), lwd=3, 
                comb_col=cbb_palette[comb_degree(m)], 
                set_order = c("Caudate", "DLPFC", "Hippocampus", "CMC DLPFC"), 
                comb_order = order(-comb_size(m)), 
                row_names_gp = gpar(fontsize = 16, fontface='bold'),
                right_annotation = right_annot, 
                top_annotation = top_annot))
od = column_order(ht)
cs = comb_size(m)
decorate_annotation("intersection_size", {
    grid.text(cs[od], x = seq_along(cs), y = unit(cs[od], "native") + 
              unit(6, "pt"), 
        default.units = "native", just = "bottom", gp = gpar(fontsize = 11))
})
dev.off()

In [ ]:
%%R
right_ha = rowAnnotation(
    "Intersection\nsize" = anno_barplot(comb_size(m), border=F,
                                        ylim = c(0, 2000), 
                                        gp=gpar(fill=cbb_palette[comb_degree(m)]),
                                        width = unit(7, "cm")))
top_ha = HeatmapAnnotation(
    "Set size" = anno_barplot(set_size(m), border=F,
                              ylim = c(0, 2000),
                              gp = gpar(fill = "black"), 
                              height = unit(2, "cm")), 
    gap = unit(2, "mm"), annotation_name_side = "left", 
    annotation_name_rot = 90)

pdf("cmc_sex_tissue_upsetR_DEgenes_transpose_maleSpecific.pdf", width=6, height=10)
ht = draw(UpSet(t(m), pt_size=unit(5, "mm"), lwd=3,
                comb_order = order(-comb_size(m)),
                comb_col=cbb_palette[comb_degree(m)], 
                set_order = c("Caudate", "DLPFC", "Hippocampus", "CMC DLPFC"), 
                column_names_gp = gpar(fontsize = 16, fontface='bold'),
                right_annotation = right_ha, top_annotation=top_ha))

od = rev(row_order(ht))
cs = comb_size(m)
decorate_annotation("Intersection\nsize", {
    grid.text(cs[od], y = seq_along(cs), x = unit(cs[od], "native") + 
              unit(6, "pt"), 
        default.units = "native", just = "left", gp = gpar(fontsize = 11))
})
dev.off()

In [ ]:
dft = pd.read_csv('../../../cmc_dlpfc/male_analysis/metrics_summary/_m/male_specific_DE_genes.txt',
                  sep='\t')
dft['Dir'] = np.sign(dft['t'])
dft.head()

In [ ]:
shared_df = dft.loc[:, ['Feature', 'ensemblID', 'Chrom', 'Symbol', 'Dir']]\
               .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                                      set(phase2_hippo['ensemblID']) & 
                                                      set(phase3_caudate['ensemblID']) & 
                                                      set(cmc['ensemblID']))}), 
                      on='ensemblID')
shared_df.to_csv('cmc_shared_degs_annotation_maleSpecific.txt', sep='\t', 
                 index=False, header=True)
shared_df

In [ ]:
cmc.merge(gtf_annot[['gene_id', 'ensemblID', 'gene_name', 'seqname']], 
            on='ensemblID')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase2_dlpfc['ensemblID']) & 
                                            set(cmc['ensemblID']))}), 
            on='ensemblID')

In [ ]:
cmc.merge(gtf_annot[['gene_id', 'ensemblID', 'gene_name', 'seqname']], 
            on='ensemblID')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase2_hippo['ensemblID']) & 
                                            set(cmc['ensemblID']))}), 
            on='ensemblID')

In [ ]:
cmc.merge(gtf_annot[['gene_id', 'ensemblID', 'gene_name', 'seqname']], 
            on='ensemblID')\
     .merge(pd.DataFrame({'ensemblID': list(set(phase3_caudate['ensemblID']) & 
                                            set(cmc['ensemblID']))}), 
            on='ensemblID')

In [ ]:
gtf_annot[["gene_id", 'ensemblID', 'gene_name', 'seqname', 'gene_type']]\
    .merge(df, left_on='ensemblID', right_index=True)\
    .to_csv('cmc_all_deg_across_tissues_maleSpecific.csv')